# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [2]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-04-18 00:20:16,191 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460953213.log


This non-commercial license of GraphLab Create is assigned to rupeshmore85@gmail.com and will expire on December 13, 2016. For commercial licensing options, visit https://dato.com/buy/.


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [4]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
train_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.040803     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [7]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [8]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [9]:
test_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


In [1]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = data['price']-predictions
    # Then square and add them up
    RSS = (residuals*residuals).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
# Error in lambda function

# train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
# test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [ ]:
train_data['bedrooms_squared'] = [x*x for x in train_data['bedrooms']]
test_data['bedrooms_squared'] = [x*x for x in test_data['bedrooms']]

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

In [ ]:
train_data['log_sqft_living'] = [log(x) for x in train_data['sqft_living']]
test_data['log_sqft_living'] = [log(x) for x in test_data['sqft_living']]

In [ ]:
train_data['lat_plus_long'] = train_data['lat']+train_data['long']
test_data['lat_plus_long'] = test_data['lat']+test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [ ]:
print round(test_data['bedrooms_squared'].mean(),2)
print round(test_data['bed_bath_rooms'].mean(),2)
print round(test_data['log_sqft_living'].mean(),2)
print round(test_data['lat_plus_long'].mean(),2)

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)
model1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
model2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)

In [ ]:
# Examine/extract each model's coefficients:
model1_weight_summary = model1.get("coefficients")
print model1_weight_summary
model2_weight_summary = model2.get("coefficients")
print model2_weight_summary
model3_weight_summary = model3.get("coefficients")
print model3_weight_summary

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
RSS1_training = get_residual_sum_of_squares(model1, train_data, train_data['price'])
print RSS1_training
RSS2_training = get_residual_sum_of_squares(model2, train_data, train_data['price'])
print RSS2_training
RSS3_training = get_residual_sum_of_squares(model3, train_data, train_data['price'])
print RSS3_training

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:
# Compute the RSS on TRAINING data for each of the three models and record the values:
RSS1 = get_residual_sum_of_squares(model1, test_data, test_data['price'])
print RSS1
RSS2 = get_residual_sum_of_squares(model2, test_data, test_data['price'])
print RSS2
RSS3 = get_residual_sum_of_squares(model3, test_data, test_data['price'])
print RSS3

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.

# Quiz 2

In [4]:
import numpy as np
import math

In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = data_sframe[features]
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray=data_sframe[output]
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [6]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [7]:
def feature_derivative(errors, feature):
    derivative = 2 * np.dot(feature, errors)
    return(derivative)

In [8]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        # compute the errors as predictions - output:
        predictions = predict_outcome(feature_matrix, weights)
        errors = predictions - output
        
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares=derivative*derivative+gradient_sum_squares
            
            # update the weight based on step size and derivative:
            weights[i] = weights[i] - step_size* derivative
#             print 'i',i,' weights: ',weights[i]
        gradient_magnitude = math.sqrt(gradient_sum_squares)
#         print gradient_magnitude
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [9]:
train_data,test_data = sales.random_split(.8,seed=0)

In [15]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [16]:
simple_feature_matrix

array([[  1.00000000e+00,   1.18000000e+03],
       [  1.00000000e+00,   2.57000000e+03],
       [  1.00000000e+00,   7.70000000e+02],
       ..., 
       [  1.00000000e+00,   1.53000000e+03],
       [  1.00000000e+00,   1.60000000e+03],
       [  1.00000000e+00,   1.02000000e+03]])

In [17]:
output

array([ 221900.,  538000.,  180000., ...,  360000.,  400000.,  325000.])

In [18]:
initial_weights

array([ -4.70000000e+04,   1.00000000e+00])

In [19]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size, tolerance)

In [20]:
# Question 1
simple_weights

array([-46999.88716555,    281.91211912])

In [21]:
len(predict_outcome(simple_feature_matrix, np.array(initial_weights)))

17384

In [22]:
len(output)

17384

In [23]:
feature_derivative((predict_outcome(simple_feature_matrix, np.array(initial_weights)) - output), simple_feature_matrix[:, 0])

-20314476464.0

In [24]:
train_data,test_data = sales.random_split(.8,seed=0)

In [25]:
# Question 2
simple_features = ['sqft_living']
my_output= 'price'
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [26]:
predict_outcome(test_simple_feature_matrix, simple_weights)

array([ 356134.44317093,  784640.86422788,  435069.83652353, ...,
        663418.65300782,  604217.10799338,  240550.4743332 ])

In [27]:
test_simple_feature_matrix

array([[  1.00000000e+00,   1.43000000e+03],
       [  1.00000000e+00,   2.95000000e+03],
       [  1.00000000e+00,   1.71000000e+03],
       ..., 
       [  1.00000000e+00,   2.52000000e+03],
       [  1.00000000e+00,   2.31000000e+03],
       [  1.00000000e+00,   1.02000000e+03]])

In [28]:
test_simple_features_model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = simple_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 1

Number of unpacked features : 1

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.037143     | 4349521.926170     | 262943.613754 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [32]:
get_residual_sum_of_squares(test_simple_features_model_1, test_data, test_data['price'])

275402888125772.38

In [33]:
# Q3:

model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(train_model2_feature_matrix, train_model2_output) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [34]:
# Create weights using train data
gradient_descent_model_features_weights = regression_gradient_descent(train_model2_feature_matrix, train_model2_output,initial_weights, \
                                                                     step_size, tolerance)

In [35]:
print gradient_descent_model_features_weights

[ -9.99999688e+04   2.45072603e+02   6.52795277e+01]


In [36]:
# Q3(test data):

model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(test_model2_feature_matrix, test_model2_output) = get_numpy_data(test_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [37]:
predict_outcome(test_model2_feature_matrix, gradient_descent_model_features_weights)

array([ 366651.41203656,  762662.39786164,  386312.09499712, ...,
        682087.39928241,  585579.27865729,  216559.20396617])

In [38]:
# which model was closer to test house 1 (model1 or model2)
# Model 1, house pred: 356,134
# Model 2, house pred: 366,651
test_data[0] # 310,000

{'bathrooms': 1.0,
 'bedrooms': 3.0,
 'condition': 4,
 'constant': 1,
 'date': datetime.datetime(2014, 5, 28, 0, 0, tzinfo=GMT +0.0),
 'floors': '1.5',
 'grade': 7,
 'id': '0114101516',
 'lat': 47.75584254,
 'long': -122.22874498,
 'price': 310000.0,
 'sqft_above': 1430,
 'sqft_basement': 0,
 'sqft_living': 1430.0,
 'sqft_living15': 1780.0,
 'sqft_lot': 19901,
 'sqft_lot15': 12697.0,
 'view': 0,
 'waterfront': 0,
 'yr_built': 1927,
 'yr_renovated': 0,
 'zipcode': '98028'}

In [39]:
test_simple_features_model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 2

Number of unpacked features : 2

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.016711     | 4588581.155968     | 261304.240828 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [40]:
get_residual_sum_of_squares(test_simple_features_model_2, test_data, test_data['price'])

270270383931775.12

In [41]:
# Q4: which model has lowest RSS:

# Model_1: 275402888125772.38
# Model_2: 270270383931775.12
